In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def weighted_divisor_sum(k,r):
    ans=0
    pfk=prime_factors(k)
    weights=[]
    for d in divisors(k):
        weight=1
        pfd=prime_factors(d)
        for p in pfd:
            opd=order(p,d)
            opk=order(p,k)
            wt=binomial(opk,opd)
            weight=weight*wt
        weights=weights+[weight]
        ans=ans+weight*d^r
    return ans

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    if n==1:return 1
    if n>1:return len(prime_factors(n))

def omega2(n):
    return len(prime_factors(n))

def bigOmega(n):
    F=factor(n)
    ans=0
    for f in F:
        ans=ans+f[1]
    return ans

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
wtDvSrs=0
for n in [1..10]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,0)*x^n
print(wtDvSrs)
print(wtDvSrs.truncate(4))

4*x^10 + 4*x^9 + 8*x^8 + 2*x^7 + 4*x^6 + 2*x^5 + 4*x^4 + 2*x^3 + 2*x^2 + x
2*x^3 + 2*x^2 + x


In [6]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,0)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run16may23no1.txt','wb') 
data=[]
start=time.time()/3600
for a in [1..1000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 6.219663191586733e-06)
(2, 0.00026075553614646196)
(3, 0.0008062302367761731)
(4, 0.0013163457624614239)
(5, 0.0020860129734501243)
(6, 0.0028608579887077212)
(7, 0.003911973035428673)
(8, 0.004657502693589777)
(9, 0.005679058260284364)
(10, 0.006675447977613658)
(11, 0.007937926915474236)
(12, 0.008926330192480236)
(13, 0.010194324073381722)
(14, 0.01145583763718605)
(15, 0.01299366046441719)
(16, 0.013939624652266502)
(17, 0.015163404925260693)
(18, 0.01637977024074644)
(19, 0.01785907690646127)
(20, 0.019071061571594328)
(21, 0.020562857680488378)
(22, 0.02205155190313235)
(23, 0.023809038219042122)
(24, 0.025015873834490776)
(25, 0.02650253742467612)
(26, 0.02800199156627059)
(27, 0.02977758046472445)
(28, 0.03124712937278673)
(29, 0.03300557273905724)
(30, 0.03476094244979322)
(31, 0.03678456740453839)
(32, 0.037957113003358245)
(33, 0.03945686295628548)
(34, 0.04088715632678941)
(35, 0.042604697693604976)
(36, 0.044042760506272316)
(37, 0.045750384393613786)
(38, 0.0474485868

(314, 0.7167470115819015)
(315, 0.7199047816102393)
(316, 0.7228022165945731)
(317, 0.7259826577501372)
(318, 0.7291519712889567)
(319, 0.7325734563637525)
(320, 0.7346490496420301)
(321, 0.7370049312594347)
(322, 0.739392876625061)
(323, 0.7421270461054519)
(324, 0.7444924921146594)
(325, 0.747116714133881)
(326, 0.7497521890909411)
(327, 0.7526493938057683)
(328, 0.7550078599597327)
(329, 0.7577623502002098)
(330, 0.76051022100728)
(331, 0.7635089463437907)
(332, 0.7662390160840005)
(333, 0.7692268691025674)
(334, 0.772215471311938)
(335, 0.7754828212782741)
(336, 0.7779436741839163)
(337, 0.7806618788163178)
(338, 0.7833939454867505)
(339, 0.7863811026909389)
(340, 0.7891092702047899)
(341, 0.7921326876967214)
(342, 0.7951266985619441)
(343, 0.7984089818783104)
(344, 0.8011342346435413)
(345, 0.804123203502968)
(346, 0.8071125313290395)
(347, 0.810370385996066)
(348, 0.8133613682584837)
(349, 0.8166209508199245)
(350, 0.8198702204390429)
(351, 0.823398684617132)
(352, 0.825891591084

(632, 1.7091539044049568)
(633, 1.7127369566587731)
(634, 1.716335378238)
(635, 1.7202039810363203)
(636, 1.7237841763417237)
(637, 1.7276614957954735)
(638, 1.7315099166007712)
(639, 1.7356373652000912)
(640, 1.7381479679606855)
(641, 1.740935246576555)
(642, 1.7437219871790148)
(643, 1.746753957180772)
(644, 1.7495399629697204)
(645, 1.7525903458008543)
(646, 1.7556351679959334)
(647, 1.7589853177196346)
(648, 1.7617673155036755)
(649, 1.7648211874184199)
(650, 1.7678734338842332)
(651, 1.7712108580162749)
(652, 1.7742688571452163)
(653, 1.7775858671520837)
(654, 1.7808987034950405)
(655, 1.7845067740418017)
(656, 1.7873084904858842)
(657, 1.7903589057386853)
(658, 1.7934243301860988)
(659, 1.796740313002374)
(660, 1.7997985701658763)
(661, 1.8031244007288478)
(662, 1.8064583157538436)
(663, 1.8100255877361633)
(664, 1.813079348532483)
(665, 1.8163932288298383)
(666, 1.8197145521407947)
(667, 1.8233260954730213)
(668, 1.826663805230055)
(669, 1.830263177107554)
(670, 1.83390161045826

(952, 2.8415101824793965)
(953, 2.8454709696234204)
(954, 2.8494755982537754)
(955, 2.853857918526046)
(956, 2.8578590491088107)
(957, 2.862121107405983)
(958, 2.8663461340474896)
(959, 2.8708875629818067)
(960, 2.8740477361134253)
(961, 2.8774743258254603)
(962, 2.8809059435734525)
(963, 2.88459021988092)
(964, 2.8880182313150726)
(965, 2.8917351795826107)
(966, 2.895444233843591)
(967, 2.899383334966842)
(968, 2.902892027166672)
(969, 2.90658673690632)
(970, 2.9102495019324124)
(971, 2.9142115102149546)
(972, 2.917916022473946)
(973, 2.9218809852027334)
(974, 2.9258829249301925)
(975, 2.9301135516143404)
(976, 2.933531738817692)
(977, 2.9372189368586987)
(978, 2.940923162677791)
(979, 2.9448828237946145)
(980, 2.9485560745815746)
(981, 2.952500554616563)
(982, 2.956459386856295)
(983, 2.9607105649774894)
(984, 2.964410528249573)
(985, 2.968375481548719)
(986, 2.9723319351905957)
(987, 2.976567999983672)
(988, 2.9805402974016033)
(989, 2.9847833149833605)
(990, 2.9890384243335575)
(99

In [9]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run16may23no2.txt','wb') 
data=[]
start=time.time()/3600
for a in [1..1000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 2.1818268578499556e-05)
(2, 0.00038815801963210106)
(3, 0.0011974394437856972)
(4, 0.0019422274781391025)
(5, 0.0030471855425275862)
(6, 0.004160958051215857)
(7, 0.005674509156960994)
(8, 0.006724553008098155)
(9, 0.008144409977830946)
(10, 0.00957836938323453)
(11, 0.01138856157194823)
(12, 0.01282882917439565)
(13, 0.014632627426180989)
(14, 0.016449568327516317)
(15, 0.01865444303257391)
(16, 0.02005571307381615)
(17, 0.021816248015966266)
(18, 0.023602236644364893)
(19, 0.02576857915846631)
(20, 0.027530589955858886)
(21, 0.02968564018374309)
(22, 0.03183843690203503)
(23, 0.03442104801069945)
(24, 0.03619992022868246)
(25, 0.03835381555836648)
(26, 0.04051746806362644)
(27, 0.04305910662515089)
(28, 0.04523812688421458)
(29, 0.04777745861792937)
(30, 0.050328464130871)
(31, 0.053305653331335634)
(32, 0.05499865801539272)
(33, 0.05708229943411425)
(34, 0.05916336632799357)
(35, 0.061633917735889554)
(36, 0.06372935528634116)
(37, 0.06619107408914715)
(38, 0.06869335193186998)


(316, 1.0334058732842095)
(317, 1.0379925855086185)
(318, 1.0425697163445875)
(319, 1.0475500674219802)
(320, 1.050653278536629)
(321, 1.0541368438862264)
(322, 1.0576133877621032)
(323, 1.0614243539166637)
(324, 1.0649012963403948)
(325, 1.0687362638418563)
(326, 1.072573708544951)
(327, 1.0767828658572398)
(328, 1.0802482899744064)
(329, 1.084082309214864)
(330, 1.0879304811242037)
(331, 1.0921356119215488)
(332, 1.0959571583080105)
(333, 1.1001503177685663)
(334, 1.104363153630402)
(335, 1.1089200696442276)
(336, 1.1123967872117646)
(337, 1.1162280305870809)
(338, 1.120089002768509)
(339, 1.124303836375475)
(340, 1.128243396698963)
(341, 1.132453281315975)
(342, 1.1366723055252805)
(343, 1.1412561038159765)
(344, 1.145099873363506)
(345, 1.1492946157814004)
(346, 1.1535120130865835)
(347, 1.1580904514412396)
(348, 1.1623091194196604)
(349, 1.1668720961315557)
(350, 1.1714633352239616)
(351, 1.1764311369042844)
(352, 1.1799494049628265)
(353, 1.1837654822156765)
(354, 1.1875965955550

(636, 2.4381178675103)
(637, 2.443423800810706)
(638, 2.4487224165932275)
(639, 2.454430343292188)
(640, 2.457933817699086)
(641, 2.4617977546877228)
(642, 2.4656424018903635)
(643, 2.469864715007134)
(644, 2.4737362693995237)
(645, 2.477936721930746)
(646, 2.4821501244441606)
(647, 2.486733495257795)
(648, 2.4905787116731517)
(649, 2.494806250790134)
(650, 2.4990127625060268)
(651, 2.5035920058144256)
(652, 2.507829488313291)
(653, 2.5124196244287305)
(654, 2.5170174530358054)
(655, 2.521951508591883)
(656, 2.5257967588840984)
(657, 2.530014799966011)
(658, 2.5342312082648277)
(659, 2.538839388813358)
(660, 2.543092909443658)
(661, 2.5476706601912156)
(662, 2.552275355788879)
(663, 2.5572344722459093)
(664, 2.5615765913389623)
(665, 2.5662597732734866)
(666, 2.570847061695531)
(667, 2.5758016263716854)
(668, 2.5803953491849825)
(669, 2.585337882512249)
(670, 2.5902957949438132)
(671, 2.595614453894086)
(672, 2.5994545077555813)
(673, 2.603683421621099)
(674, 2.607891088584438)
(675, 2

(958, 4.022117083019111)
(959, 4.02840497775469)
(960, 4.032847843889613)
(961, 4.037684228271246)
(962, 4.042560877802316)
(963, 4.047785724746063)
(964, 4.052587641694117)
(965, 4.057779118069448)
(966, 4.063013375853188)
(967, 4.068522538582329)
(968, 4.073331400228199)
(969, 4.078514979162719)
(970, 4.083704263321124)
(971, 4.0892743724398315)
(972, 4.094454416655935)
(973, 4.100045966042671)
(974, 4.105597202433273)
(975, 4.111474551376887)
(976, 4.1162683852599)
(977, 4.121461250819266)
(978, 4.126697946048807)
(979, 4.132239693833981)
(980, 4.137426023604348)
(981, 4.142967758583836)
(982, 4.148537352215499)
(983, 4.154473574773874)
(984, 4.15967973304214)
(985, 4.165272353624459)
(986, 4.170844116655644)
(987, 4.176765385782346)
(988, 4.182315371348523)
(989, 4.18828423024388)
(990, 4.194228864100296)
(991, 4.200504278589506)
(992, 4.20533912855899)
(993, 4.210566217487212)
(994, 4.215761953033507)
(995, 4.221327679930255)
(996, 4.226514110807329)
(997, 4.23208059836179)
(998, 

In [ ]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run17may23no2.txt','wb') 
data=[]
start=time.time()/3600
for a in [1001..5000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1001, 0.005657506757415831)
(1002, 0.011293448333162814)
(1003, 0.017304865876212716)
(1004, 0.022972901409957558)
(1005, 0.028976259753108025)
(1006, 0.035003912867978215)
(1007, 0.04140946170082316)
(1008, 0.04671198362484574)
(1009, 0.05234624951845035)
(1010, 0.05806720338296145)
(1011, 0.06404783995822072)
(1012, 0.06964777945540845)
(1013, 0.07567315141204745)
(1014, 0.08168046525679529)
(1015, 0.08805653837043792)
(1016, 0.0936420692014508)
(1017, 0.09963171975687146)
(1018, 0.10558945749653503)
(1019, 0.1119019414181821)
(1020, 0.11787472700234503)
(1021, 0.12437761586625129)
(1022, 0.1307638980797492)
(1023, 0.13790455309208483)
(1024, 0.1415274061728269)
(1025, 0.1454757725005038)
(1026, 0.14946297474671155)
(1027, 0.1538304919959046)
(1028, 0.1577861924888566)
(1029, 0.16210888721980155)
(1030, 0.1664471714757383)
(1031, 0.17109733109828085)
(1032, 0.17505356506444514)
(1033, 0.17933370999526232)
(1034, 0.18364008550997823)
(1035, 0.1882810500683263)
(1036, 0.19258688972331

(1304, 1.5491116336197592)
(1305, 1.554204359475989)
(1306, 1.55931349581806)
(1307, 1.564818031154573)
(1308, 1.5699341189465486)
(1309, 1.5754379272693768)
(1310, 1.5808965119649656)
(1311, 1.5867335686343722)
(1312, 1.5910972880665213)
(1313, 1.5958361369557679)
(1314, 1.6005755144287832)
(1315, 1.605662080575712)
(1316, 1.6104062083759345)
(1317, 1.6155190194258466)
(1318, 1.620700708066579)
(1319, 1.6262037486885674)
(1320, 1.6310149861383252)
(1321, 1.6361517158802599)
(1322, 1.641276408918202)
(1323, 1.6467432503122836)
(1324, 1.651856177195441)
(1325, 1.6573488308349624)
(1326, 1.662869656109251)
(1327, 1.6688158577890135)
(1328, 1.6735609730822034)
(1329, 1.6786795986699872)
(1330, 1.6838133983546868)
(1331, 1.6892853105673566)
(1332, 1.6944008613936603)
(1333, 1.6999243011814542)
(1334, 1.7054001563810743)
(1335, 1.711259460018482)
(1336, 1.7163950120448135)
(1337, 1.7218743478297256)
(1338, 1.7273734246846288)
(1339, 1.733221963106189)
(1340, 1.738702029979322)
(1341, 1.7445

In [3]:
len(data)
# cat walked across the keyboard, halted the output w/o printing last several results.

629

In [4]:
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [6]:
import pickle
rfile=open('/Users/barrybrent/run17may23no2.txt','rb') 
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ls=len(s)
print(ls)
pair=s[ls-1]
a=pair[0]
print(a)

629
1629


In [2]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run17may23no3.txt','wb') 
data=[]
start=time.time()/3600
for a in [1630..5000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1630, 0.005862213904038072)
(1631, 0.012113302480429411)
(1632, 0.016933073406107724)
(1633, 0.022228178626392037)
(1634, 0.02739683008985594)
(1635, 0.032923941675107926)
(1636, 0.03812851005932316)
(1637, 0.043644219753332436)
(1638, 0.049174240906722844)
(1639, 0.0550834039459005)
(1640, 0.06031674053519964)
(1641, 0.065862545336131)
(1642, 0.07136222388362512)
(1643, 0.07725970505271107)
(1644, 0.0828186683356762)
(1645, 0.08875320554943755)
(1646, 0.0946489714551717)
(1647, 0.1009042183868587)
(1648, 0.1060786972520873)
(1649, 0.11162935692118481)
(1650, 0.11721869703615084)
(1651, 0.1231615903088823)
(1652, 0.12869080860400572)
(1653, 0.134709763340652)
(1654, 0.14066190499579534)
(1655, 0.1468729845364578)
(1656, 0.15239547775126994)
(1657, 0.15832157974364236)
(1658, 0.16418798867380247)
(1659, 0.1705521655967459)
(1660, 0.17645271198125556)
(1661, 0.18273294891696423)
(1662, 0.18895588669693097)
(1663, 0.19559510389808565)
(1664, 0.20008171192603186)
(1665, 0.2048510677996091

(1933, 1.7650747697334737)
(1934, 1.771082733408548)
(1935, 1.7774489522562362)
(1936, 1.782715906680096)
(1937, 1.788369621441234)
(1938, 1.7939724555471912)
(1939, 1.7999265886028297)
(1940, 1.8055599630461074)
(1941, 1.8115677750320174)
(1942, 1.8175440486520529)
(1943, 1.8238796147634275)
(1944, 1.8294572744634934)
(1945, 1.8354185527423397)
(1946, 1.8414174102945253)
(1947, 1.847787182254251)
(1948, 1.853789342509117)
(1949, 1.8601804019417614)
(1950, 1.8664738758234307)
(1951, 1.8734625697834417)
(1952, 1.8787996575701982)
(1953, 1.8844427288859151)
(1954, 1.8900890919030644)
(1955, 1.8960800353088416)
(1956, 1.9017196397762746)
(1957, 1.9076732572866604)
(1958, 1.9136538208113052)
(1959, 1.9200343642150983)
(1960, 1.925688904186245)
(1961, 1.9317086130613461)
(1962, 1.9376820325269364)
(1963, 1.9440512888832018)
(1964, 1.9500644214567728)
(1965, 1.9564177394495346)
(1966, 1.9628080341499299)
(1967, 1.9695947672589682)
(1968, 1.975251063646283)
(1969, 1.981272346980404)
(1970, 1.

(2242, 3.5225409230333753)
(2243, 3.5280491338926367)
(2244, 3.533199051686097)
(2245, 3.538678282580804)
(2246, 3.544184717524331)
(2247, 3.5500562025699764)
(2248, 3.5552795623079874)
(2249, 3.56079943140503)
(2250, 3.5663220583228394)
(2251, 3.5722521753050387)
(2252, 3.577778831415344)
(2253, 3.583705433353316)
(2254, 3.589583329448942)
(2255, 3.595856744213961)
(2256, 3.601011543069035)
(2257, 3.606539440632332)
(2258, 3.6120615222025663)
(2259, 3.617938358685933)
(2260, 3.623476095846854)
(2261, 3.629442165547516)
(2262, 3.6353297627647407)
(2263, 3.6415897363913245)
(2264, 3.6471853436669335)
(2265, 3.653053505870048)
(2266, 3.6589662614278495)
(2267, 3.6652721388963982)
(2268, 3.671183193626348)
(2269, 3.677450016431976)
(2270, 3.683723960071802)
(2271, 3.690344423055649)
(2272, 3.6955000266898423)
(2273, 3.70099209388718)
(2274, 3.706522617547307)
(2275, 3.7125253753038123)
(2276, 3.7182974361348897)
(2277, 3.724230246152729)
(2278, 3.7302085430710576)
(2279, 3.736493282776791

(2556, 5.399023572797887)
(2557, 5.406337001128122)
(2558, 5.413551791396458)
(2559, 5.421113354153931)
(2560, 5.425481880607549)
(2561, 5.430308740003966)
(2562, 5.435150184202939)
(2563, 5.440397021127865)
(2564, 5.445253392797895)
(2565, 5.450469490897376)
(2566, 5.455826219462324)
(2567, 5.461446468660142)
(2568, 5.466305332200136)
(2569, 5.4715264336555265)
(2570, 5.476751199166756)
(2571, 5.482326136436313)
(2572, 5.487634921388235)
(2573, 5.493244201410562)
(2574, 5.498800363857299)
(2575, 5.504759583855048)
(2576, 5.509706868673675)
(2577, 5.514977566082962)
(2578, 5.52022130478872)
(2579, 5.525822609139141)
(2580, 5.531029394478537)
(2581, 5.536615356686525)
(2582, 5.542173409485258)
(2583, 5.548193387279753)
(2584, 5.553424689453095)
(2585, 5.559000712237321)
(2586, 5.564598085300531)
(2587, 5.570554303063545)
(2588, 5.5761346577783115)
(2589, 5.582088093622588)
(2590, 5.588058763358276)
(2591, 5.594382109469734)
(2592, 5.599218677030876)
(2593, 5.604453019797802)
(2594, 5.60

(2872, 7.29410708945943)
(2873, 7.300518358068075)
(2874, 7.30694342777133)
(2875, 7.313698368438054)
(2876, 7.320112485031132)
(2877, 7.326994946750347)
(2878, 7.3337790663936175)
(2879, 7.340919649985153)
(2880, 7.34626824920997)
(2881, 7.351961123931687)
(2882, 7.357644288102165)
(2883, 7.363749799784273)
(2884, 7.369456990272738)
(2885, 7.375502007256728)
(2886, 7.3815523617086)
(2887, 7.3880397697212175)
(2888, 7.393720140797086)
(2889, 7.39977946144063)
(2890, 7.40584744309308)
(2891, 7.412213788891677)
(2892, 7.418229801405687)
(2893, 7.4246497477870435)
(2894, 7.4310776180936955)
(2895, 7.437861142738257)
(2896, 7.443549838848412)
(2897, 7.449591084732674)
(2898, 7.455656430334784)
(2899, 7.462069846398663)
(2900, 7.468081230879761)
(2901, 7.474530260835309)
(2902, 7.4809550900245085)
(2903, 7.4877196558518335)
(2904, 7.493769296968821)
(2905, 7.500184693955816)
(2906, 7.50661644974025)
(2907, 7.513401399482973)
(2908, 7.5198259659227915)
(2909, 7.526587593602017)
(2910, 7.5333

...

(3500, 11.221563731727656)
(3501, 11.22834378364496)
(3502, 11.235213041945826)
(3503, 11.242406715347897)
(3504, 11.248510962177534)
(3505, 11.255016787501518)
(3506, 11.261470003053546)
(3507, 11.26829253725009)
(3508, 11.274795027798973)
(3509, 11.281564966135193)
(3510, 11.288394686125685)
(3511, 11.295596494455822)
(3512, 11.30205404694425)
(3513, 11.308952933119144)
(3514, 11.315877181710675)
(3515, 11.323130142816808)
(3516, 11.330027970019728)
(3517, 11.337321064202115)
(3518, 11.344568047556095)
(3519, 11.352222351939417)
(3520, 11.35803826112533)
(3521, 11.364206271187868)
(3522, 11.370414500299376)
(3523, 11.376906883670017)
(3524, 11.383067962538917)
(3525, 11.38976037449902)
(3526, 11.396302426117472)
(3527, 11.403195495309774)
(3528, 11.40937688224949)
(3529, 11.415894535603002)
(3530, 11.42242830368923)
(3531, 11.429348784731701)
(3532, 11.435867735825013)
(3533, 11.4427464150358)
(3534, 11.449640601989813)
(3535, 11.456891398644075)
(3536, 11.463051162543707)
(3537, 11.

(3806, 13.213463217834942)
(3807, 13.221130322490353)
(3808, 13.227311925555114)
(3809, 13.233829791366588)
(3810, 13.240391158673447)
(3811, 13.247307456680574)
(3812, 13.253847041924018)
(3813, 13.260760990902781)
(3814, 13.267660829762463)
(3815, 13.274928966420703)
(3816, 13.281460025231354)
(3817, 13.288391295005567)
(3818, 13.295237485552207)
(3819, 13.302530922810547)
(3820, 13.309424585255329)
(3821, 13.316639806726016)
(3822, 13.323939214169513)
(3823, 13.33160045556724)
(3824, 13.33815182338003)
(3825, 13.345055365585722)
(3826, 13.351950288109947)
(3827, 13.359241603349801)
(3828, 13.366150029178243)
(3829, 13.373432561988011)
(3830, 13.380713086109608)
(3831, 13.388380484480876)
(3832, 13.395317318383604)
(3833, 13.402549182239454)
(3834, 13.409833376354072)
(3835, 13.417470580898225)
(3836, 13.424711273342837)
(3837, 13.432354618911631)
(3838, 13.439968874445185)
(3839, 13.447959088662174)
(3840, 13.453312150319107)
(3841, 13.459103479166515)
(3842, 13.46489958447637)
(384

(4113, 15.330974988348316)
(4114, 15.336159222817514)
(4115, 15.341757875867188)
(4116, 15.346970630576834)
(4117, 15.35252922528889)
(4118, 15.358108827786054)
(4119, 15.364043117850088)
(4120, 15.369231794436928)
(4121, 15.374812194437254)
(4122, 15.380358185560908)
(4123, 15.386319264420308)
(4124, 15.391939519206062)
(4125, 15.397857873991597)
(4126, 15.403783602523617)
(4127, 15.410082371730823)
(4128, 15.414908847247716)
(4129, 15.42013097560266)
(4130, 15.425332397804596)
(4131, 15.430891526688356)
(4132, 15.436099297832698)
(4133, 15.441666725033429)
(4134, 15.447255562525243)
(4135, 15.453180986689404)
(4136, 15.45839061588049)
(4137, 15.463957449188456)
(4138, 15.469545138359535)
(4139, 15.475503109686542)
(4140, 15.48105113231577)
(4141, 15.486971573904157)
(4142, 15.492887299449649)
(4143, 15.49914575368166)
(4144, 15.504336607817095)
(4145, 15.509919146134052)
(4146, 15.515481925569475)
(4147, 15.521439118136186)
(4148, 15.526969827245921)
(4149, 15.5328824261087)
(4150, 1

(4421, 17.19771888141986)
(4422, 17.20372702385066)
(4423, 17.210095551330596)
(4424, 17.21576231229119)
(4425, 17.221848533081356)
(4426, 17.227835451078136)
(4427, 17.234225859690923)
(4428, 17.24025738524506)
(4429, 17.246678088908084)
(4430, 17.25309590587858)
(4431, 17.25988449086435)
(4432, 17.265538414474577)
(4433, 17.27155909501016)
(4434, 17.277575477259234)
(4435, 17.283972387493122)
(4436, 17.29001831507776)
(4437, 17.296436666103546)
(4438, 17.30283057811903)
(4439, 17.30960266641341)
(4440, 17.315624291892163)
(4441, 17.322014320874587)
(4442, 17.328410105255898)
(4443, 17.335195039690007)
(4444, 17.34161243500421)
(4445, 17.348363526398316)
(4446, 17.355136031634174)
(4447, 17.362270792538766)
(4448, 17.36794688226655)
(4449, 17.374031739193015)
(4450, 17.380041042517405)
(4451, 17.386516974773258)
(4452, 17.392607031739317)
(4453, 17.398977290082257)
(4454, 17.405330465582665)
(4455, 17.412113839178346)
(4456, 17.418163515802007)
(4457, 17.424557916645426)
(4458, 17.430

(4729, 19.182571131386794)
(4730, 19.1893414358492)
(4731, 19.196473150863312)
(4732, 19.203243685828056)
(4733, 19.21038117277203)
(4734, 19.217494793934748)
(4735, 19.225005706422962)
(4736, 19.230316319444682)
(4737, 19.235996481438633)
(4738, 19.241679904109333)
(4739, 19.247698642197065)
(4740, 19.253392197773792)
(4741, 19.2594143585884)
(4742, 19.265444925636984)
(4743, 19.2718584134127)
(4744, 19.27753388142446)
(4745, 19.283568656654097)
(4746, 19.28959176054923)
(4747, 19.295983490883373)
(4748, 19.302034447551705)
(4749, 19.308419073116966)
(4750, 19.31479781703092)
(4751, 19.321570904459804)
(4752, 19.32722300972091)
(4753, 19.33321737998631)
(4754, 19.339283338573296)
(4755, 19.345677138655446)
(4756, 19.351713407493662)
(4757, 19.35813026118558)
(4758, 19.36455909453798)
(4759, 19.371347753622103)
(4760, 19.37738075782545)
(4761, 19.383794826979283)
(4762, 19.39020336669637)
(4763, 19.396974693925586)
(4764, 19.403344809194095)
(4765, 19.41012708976632)
(4766, 19.41690701